### Structural data

continous (wind speed)

discrete (count of occruece)

Categorical (fix set of value)

    Binary data (0/1)

    ordinal data (ordered, e.g. 1, 2, 3, 4, 5)

In [ ]:
#%%R
# packages needed for chapter 1
list.of.packages <- c("dplyr", "tidyr", "ggplot2", "vioplot", "ascii", "corrplot", "descr", "matrixStats", "hexbin")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

In [ ]:
# %%R
# packages needed for chapter 1
library(dplyr)
library(tidyr)
library(ggplot2)
library(vioplot)
library(ascii)
library(corrplot)
library(descr)
library("matrixStats")
library("hexbin")

In [ ]:
# %%R
# Import the datasets needed for chapter 1
PSDS_PATH <- file.path('~/Desktop', 'statistics-for-data-scientists')
dir.create(file.path(PSDS_PATH, 'figures'))

state <- read.csv(file.path(PSDS_PATH, 'data', 'state.csv'))
dfw <- read.csv(file.path(PSDS_PATH, 'data', 'dfw_airline.csv'))
sp500_px <- read.csv(file.path(PSDS_PATH, 'data', 'sp500_px.csv'))
sp500_sym <- read.csv(file.path(PSDS_PATH, 'data', 'sp500_sym.csv'), stringsAsFactors = FALSE)
kc_tax <- read.csv(file.path(PSDS_PATH, 'data', 'kc_tax.csv'))
lc_loans <- read.csv(file.path(PSDS_PATH, 'data', 'lc_loans.csv'))
airline_stats <- read.csv(file.path(PSDS_PATH, 'data', 'airline_stats.csv'), stringsAsFactors = FALSE)
airline_stats$airline <- ordered(airline_stats$airline, levels=c('Alaska', 'American', 'Jet Blue', 'Delta', 'United', 'Southwest'))

### Rectangular Data

rows - cases (example: different states)

columns - features

In [ ]:
## Code to create state table
state_asc <- state
state_asc[["Population"]] <- formatC(state_asc[["Population"]], format="d", digits=0, big.mark=",")
ascii(state_asc[1:8,], digits=c(0, 0,1), align=c("l", "l", "r", "r"), caption="A few rows of the +data.frame state+ of population and murder rate by state.")

### mean

trim: fraction of (0 to 0.5) to be trimmed from each end.
Values outside with take the value at the trimmed point.


### median

The value such that one-half of the data lies above and below (50th percentile)

In [ ]:
## Code snippet 1.1
mean(state[["Population"]])
mean(state[["Population"]], trim=0.1)
median(state[["Population"]])

In [ ]:
## Code snippet 1.2
mean(state[["Murder.Rate"]])
weighted.mean(state[["Murder.Rate"]], w=state[["Population"]])

### Standard deviation

The square root of the variance

### Median absolute deviation from the median

The median of the absolute values of the deviations from the 
median.

This definition is robust to outliers and extreme values
(while the variance and the sd are not.)

### percentile

For example, to find the 80th percentile, sort the data.
Then, starting with the smallest value, proceed 80 percent of the way to the largest
value

### Interquartile range

The difference between the 75th percentile and the 25th percentile (IQR)

In [ ]:
## Code snippet 1.3
sd(state[["Population"]])
IQR(state[["Population"]])
mad(state[["Population"]])

It is common to report the quartiles (25th, 50th, and 75th percentiles)
and the deciles (the 10th, 20th, …, 90th percentiles).

In [ ]:
## Code snippet 1.4
quantile(state[["Murder.Rate"]], p=c(.05, .25, .5, .75, .95))

In [ ]:
## Code to create PercentileTable
ascii(
  quantile(state[["Murder.Rate"]], p=c(.05, .25, .5, .75, .95)),
  include.rownames=FALSE, include.colnames=TRUE, digits=rep(2,5), align=rep("r", 5), 
  caption="Percentiles of murder rate by state.")

### Boxplot

The top and bottom of the box are the 75th and 25th percentiles, respectively. 

The median is shown by the horizontal line in the box. The dashed lines, referred to as whiskers

In [ ]:
## Code snippet 1.5
boxplot(state[["Population"]]/1000000, ylab="Population (millions)")
## Code for Figure 2
png(filename=file.path(PSDS_PATH, "figures", "psds_0102.png"), width = 3, height=4, units='in', res=300)
par(mar=c(0,4,0,0)+.1)
boxplot(state[["Population"]]/1000000, ylab="Population (millions)")

### Frequency tables

A frequency table of a variable divides up the variable range into equally spaced segments and tells us how many values fall within each segment

bin size: big enough to not be granular and small enough to preserve important features

(the "seq" is like the "linspace" function in python)

In [ ]:
## Code snippet 1.6
breaks <- seq(from=min(state[["Population"]]), to=max(state[["Population"]]), length=11)
pop_freq <- cut(state[["Population"]], breaks=breaks, right=TRUE, include.lowest = TRUE)
state['PopFreq'] <- pop_freq
table(pop_freq)

In [ ]:
## Code for FreqTable
state_abb <- state %>%
  arrange(Population) %>%
  group_by(PopFreq) %>%
  summarize(state = paste(Abbreviation, collapse=","), .drop=FALSE) %>%
  complete(PopFreq, fill=list(state='')) %>%
  select(state) 

state_abb <- unlist(state_abb)

lower_br <- formatC(breaks[1:10], format="d", digits=0, big.mark=",")
upper_br <- formatC(c(breaks[2:10]-1, breaks[11]), format="d", digits=0, big.mark=",")

pop_table <- data.frame("BinNumber"=1:10,
                        "BinRange"=paste(lower_br, upper_br, sep="-"),
                        "Count"=as.numeric(table(pop_freq)),
                        "States"=state_abb)
ascii(pop_table, include.rownames=FALSE, digits=c(0, 0, 0, 0), align=c("l", "r", "r", "l"), 
      caption="A frequency table of population by state.")

### A histogram

To visualize a frequence table

Bins on the x-axis and data count on the y-axis

In [ ]:
## Code snippet 1.7
hist(state[["Population"]], breaks=breaks)
## Code for Figure 3
png(filename=file.path(PSDS_PATH, "figures", "psds_0103.png"),  width = 4, height=4, units='in', res=300)
par(mar=c(4,4,0,0)+.1)
pop_hist <- hist(state[["Population"]], breaks=breaks,
                 xlab="Population", main="")
dev.off()

### Density Plots

Density plots are different from hist plots in that they are continous distribution

First, plotting the histogram as a proportion (by "freq=FALSE" option).

Then derive the density line.

calculate the area below two points on the x-axis instead of counts in the bin.
Note that the total area under the density curve = 1,

Different functions used: over 20 R packages have offered a different solution.

In [ ]:
## Code snippet 1.8
hist(state[["Murder.Rate"]], freq=FALSE )
lines(density(state[["Murder.Rate"]]), lwd=3, col="blue")
## Code for Figure 4
png(filename=file.path(PSDS_PATH, "figures", "psds_0104.png"),  width = 4, height=4, units='in', res=300)
par(mar=c(4,4,0,0)+.1)
hist(state[["Murder.Rate"]], freq=FALSE, xlab="Murder Rate (per 100,000)", main="" )
lines(density(state[["Murder.Rate"]]), lwd=3, col="blue")
dev.off()

### Percentage of delayed flights by cause

#### matrix

it has data in 2 dimention and the names for cols and cols

as.maxtrix: convert to matrix.

coercing a vetor: produces a 1-d matrix and promotes the name of the vector as the rownames 

### bar chart

bar chart: different categories, not continous

histogram: continous

### Pie chart

alternative to the bar chart

experts think they are less informative


In [ ]:
## Code for AirportDelays
ascii(
  100*as.matrix(dfw/sum(dfw)),
  include.rownames=FALSE, include.colnames=TRUE, digits=rep(2,5), align=rep("r", 5), 
  caption="Percentage of delays by cause at Dallas-Ft. Worth airport.")

barplot(as.matrix(dfw)/6, cex.axis = 0.8, cex.names = 0.7)
## Code for figure 5
png(filename=file.path(PSDS_PATH, "figures", "psds_0105.png"),  width = 4, height=4, units='in', res=300)
par(mar=c(4, 4, 0, 1) + .1)
dev.off()


### Expectation

### Probability

From odd to probability

if the odds that a team will win are 2 to 1, its probability of winning is 2/(2+1)
= 2/3

In [ ]:
## Code for CorrTable (Table 1.7)
telecom <- sp500_px[, sp500_sym[sp500_sym$sector=="telecommunications_services", 'symbol']]
telecom <- telecom[row.names(telecom)>"2012-07-01", ]
telecom_cor <- cor(telecom)
ascii(telecom_cor, digits=c( 3,3,3,3,3), align=c("l", "r", "r", "r", "r", "r"), caption="Correlation between telecommunication stock returns.",
      include.rownames = TRUE, include.colnames = TRUE)

### correlation coefficient

![](./01.png)

The range of this value is in [-1, 1]

(operator "$" uses the second variable as index for the first one.)

serves to determine which component of the "sp500_sym" object is "etf".

the correlation coefficient is sensitive to outliers in the data

The R package robust uses the function covRob to compute a robust estimate.

Other ways to compute the correlation estimates

    Spearman's rho and Kendall's tau - uses rank and robust to outliers

In [ ]:
## Code snippet 1.10
etfs <- sp500_px[row.names(sp500_px)>"2012-07-01", 
                 sp500_sym[sp500_sym$sector=="etf", 'symbol']]
corrplot(cor(etfs), method = "ellipse")

## Code for figure 6
png(filename=file.path(PSDS_PATH, "figures", "psds_0106.png"), width = 4, height=4, units='in', res=300)
etfs <- sp500_px[row.names(sp500_px)>"2012-07-01", sp500_sym[sp500_sym$sector=="etf", 'symbol']]
library(corrplot)
corrplot(cor(etfs), method = "ellipse")
dev.off()

### The scatter plot

visualize the relation between two variables

The x-axis represents one variable and the y-axis another

(With this, we don't compute the correlation but see if there is a trend)

upper-right and lower left: two variable have same trending

upper-left and lower-right: one goes up and another goes down

(So the ellipse plot is a version of what would be in the scatter plot)

In [ ]:
## Code snippet 1.11
plot(telecom$T, telecom$VZ, xlab="T", ylab="VZ")

## Code for Figure 7
png(filename=file.path(PSDS_PATH, "figures", "psds_0107.png"),  width = 4, height=4, units='in', res=300)
par(mar=c(4,4,0,1)+.1)
plot(telecom$T, telecom$VZ, xlab="T", ylab="VZ", cex=.8)
abline(h=0, v=0, col="grey")
dev.off()

### Hexagonal binning and contours

large dataset, scatter plot is not helpful anymore.

relation between the squre feet and tax-assessed value

dark cloud is shown instead of point, furthermore, it's regrouped into hexagonal showing counts in each one.

results:

* positve relation
* additional band above: home with higher tax assessed value

The "stat_binhex" function divides reggular plane into regular hexagons, counts the number of cases in each and then maps the sum.

The "subset" is to remove the outliers and extract a subset of the dataset.

In [ ]:
## Code snippet 1.12
kc_tax0 <- subset(kc_tax, TaxAssessedValue < 750000 & SqFtTotLiving>100 &
                  SqFtTotLiving<3500)
nrow(kc_tax0)

In [ ]:
## Code snippet 1.13
ggplot(kc_tax0, (aes(x=SqFtTotLiving, y=TaxAssessedValue))) + 
  stat_binhex(colour="white") + 
  theme_bw() + 
  scale_fill_gradient(low="white", high="black") +
  labs(x="Finished Square Feet", y="Tax Assessed Value")

## Code for figure 8
png(filename=file.path(PSDS_PATH, "figures", "psds_0108.png"),  width = 4, height=4, units='in', res=300)
ggplot(kc_tax0, (aes(x=SqFtTotLiving, y=TaxAssessedValue))) + 
  stat_binhex(colour="white") + 
  theme_bw() + 
  scale_fill_gradient(low="white", high="black") +
  labs(x="Finished Square Feet", y="Tax Assessed Value")
dev.off()

The contour plot

The "peak" on the contours indicate bands of the feature

The "geom_density2d" function of ggplot2 is used to generate the plot.

In [ ]:
## Code snippet 1.14
ggplot(kc_tax0, aes(SqFtTotLiving, TaxAssessedValue)) +
  theme_bw() + 
  geom_point( alpha=0.1) + 
  geom_density2d(colour="white") + 
  labs(x="Finished Square Feet", y="Tax Assessed Value")

## Code for figure 9

png(filename=file.path(PSDS_PATH, "figures", "psds_0109.png"),  width = 4, height=4, units='in', res=300)
ggplot(kc_tax0, aes(SqFtTotLiving, TaxAssessedValue)) +
  theme_bw() + 
  geom_point(colour="blue", alpha=0.1) + 
  geom_density2d(colour="white") + 
  labs(x="Finished Square Feet", y="Tax Assessed Value")
dev.off()

### Contingency table

To summarize two categorical variables

shows the contingency table between the grade of a per
sonal loan and the outcome of that loan

The object "lc_loans" has too variables, one is the "grade" and the other is the "status" (fully paid, late, current, etc.)

In [ ]:
## Code snippet 1.15

## Code for CrossTabs
x_tab <- CrossTable(lc_loans$grade, lc_loans$status, 
                    prop.c=FALSE, prop.chisq=FALSE, prop.t=FALSE)

tots <- cbind(row.names(x_tab$tab), format(cbind(x_tab$tab, x_tab$rs)))
props <- cbind("", format(cbind(x_tab$prop.row, x_tab$rs/x_tab$gt), digits=1))
c_tot <- c("Total", format(c(x_tab$cs, x_tab$gt)))

asc_tab <- matrix(nrow=nrow(tots)*2+1, ncol=ncol(tots))
colnames(asc_tab) <- c("Grade", colnames(x_tab$tab), "Total")
idx <- seq(1, nrow(asc_tab)-1, by=2)
asc_tab[idx,] <- tots
asc_tab[idx+1,] <- props
asc_tab[nrow(asc_tab), ] <- c_tot

ascii(asc_tab,  align=c("l", "r", "r", "r", "r"), include.rownames = FALSE, include.colnames = TRUE)

### Categorical data and box plot

"boxplot" method could group numerical data with categorical data aand plot for each category.

In [ ]:
## Code snippet 1.16
boxplot(pct_carrier_delay ~ airline, data=airline_stats, ylim=c(0,50))

## Code for figure 10
png(filename=file.path(PSDS_PATH, "figures", "psds_0110.png"), width = 4, height=4, units='in', res=300)
par(mar=c(4,4,0,0)+.1)
boxplot(pct_carrier_delay ~ airline, data=airline_stats, ylim=c(0,50), cex.axis=.6,
        ylab="Daily % of Delayed Flights")

dev.off()

### The violin plot

It flips the density and plot it horizontally.

pro: it can show nuances in the distribution that aren’t perceptible in a boxplot

con: the boxplot more clearly shows the outliers in the data

In [ ]:
## Code snippet 1.17

ggplot(data=airline_stats, aes(airline, pct_carrier_delay))  + 
  ylim(0, 50) + 
  geom_violin() +
  labs(x="", y="Daily % of Delayed Flights")

## Code for figure 11
png(filename=file.path(PSDS_PATH, "figures", "psds_0111.png"), width = 4, height=4, units='in', res=300)

ggplot(data=airline_stats, aes(airline, pct_carrier_delay)) + 
  ylim(0, 50) + 
  geom_violin(draw_quantiles = c(.25, .5, .75), linetype=2) +
  geom_violin(fill=NA, size=1.1) +
  theme_bw() + 
  labs(x="", y="% of Delayed Flights")

dev.off()

### multiple category & correlation

concept of "facet": a conditioning variable to wrap the data with
(in this case, zip code)

visualizing the different bands observed in previous plots.

In [ ]:
## Code snippet 1.18

ggplot(subset(kc_tax0, ZipCode %in% c(98188, 98105, 98108, 98126)),
         aes(x=SqFtTotLiving, y=TaxAssessedValue)) + 
  stat_binhex(colour="white") + 
  theme_bw() + 
  scale_fill_gradient( low="white", high="black") +
  labs(x="Finished Square Feet", y="Tax Assessed Value") +
  facet_wrap("ZipCode")

## Code for figure 12
png(filename=file.path(PSDS_PATH, "figures", "psds_0112.png"), width = 5, height=4, units='in', res=300)

ggplot(subset(kc_tax0, ZipCode %in% c(98188, 98105, 98108, 98126)),
       aes(x=SqFtTotLiving, y=TaxAssessedValue)) + 
  stat_binhex(colour="white") + 
  theme_bw() + 
  scale_fill_gradient( low="gray95", high="blue") +
  labs(x="Finished Square Feet", y="Tax Assessed Value") +
  facet_wrap("ZipCode")
dev.off()